<a href="https://colab.research.google.com/github/YKochura/cv-kpi/blob/main/tutor/cnn/fcn-vs-cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   Author: François Chollet

## Fully connected NN

In [1]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


## Let’s look at the training data:

In [2]:
train_images.shape

(60000, 28, 28)

In [3]:
len(train_labels)

60000

In [4]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

## And here’s the test data:

In [5]:
test_images.shape

(10000, 28, 28)

In [6]:
len(test_labels)

10000

In [7]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

## The network architecture

In [8]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential([layers.Dense(512, activation="relu"),
                          layers.Dense(10, activation="softmax")
])

## The compilation step

In [9]:
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

## Preparing the image data

In [10]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

## “Fitting” the model

In [11]:
model.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 10s 18ms/step - loss: 0.2562 - accuracy: 0.9260
Epoch 2/5
469/469 [==============================] - 8s 17ms/step - loss: 0.1039 - accuracy: 0.9693
Epoch 3/5
469/469 [==============================] - 8s 17ms/step - loss: 0.0681 - accuracy: 0.9804
Epoch 4/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0500 - accuracy: 0.9853
Epoch 5/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0374 - accuracy: 0.9886


## Using the model to make predictions

In [12]:
test_digits = test_images[0:10]

In [13]:
predictions = model.predict(test_digits)

In [14]:
predictions[0]

array([1.0503297e-08, 1.6981208e-10, 1.0945527e-06, 4.9914361e-06,
       5.3762252e-13, 1.0174515e-08, 2.9959162e-14, 9.9999392e-01,
       1.0400862e-09, 5.2457917e-08], dtype=float32)

This first test digit has the highest probability score (0.9999969, almost 1) at index 7, so according to our model, it must be a 7:

In [15]:
predictions[0].argmax()

7

In [16]:
predictions[0][7]

0.9999939

In [17]:
test_labels[0]

7

On average, how good is our model at classifying such never-before-seen digits? Let’s check by computing average accuracy over the entire test set.

In [18]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.0706 - accuracy: 0.9784


In [19]:
print(f"test_acc: {test_acc}")

test_acc: 0.9783999919891357


---------------------------------

# Simple Convolutional NN

In [20]:
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [21]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 128)         73856 

In [22]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 57s 60ms/step - loss: 0.1577 - accuracy: 0.9514
Epoch 2/5
938/938 [==============================] - 57s 60ms/step - loss: 0.0460 - accuracy: 0.9861
Epoch 3/5
938/938 [==============================] - 55s 59ms/step - loss: 0.0312 - accuracy: 0.9905
Epoch 4/5
938/938 [==============================] - 56s 60ms/step - loss: 0.0242 - accuracy: 0.9928
Epoch 5/5
938/938 [==============================] - 55s 59ms/step - loss: 0.0184 - accuracy: 0.9945


In [23]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print(f"Test accuracy: {test_acc:.3f}")

313/313 [==============================] - 3s 9ms/step - loss: 0.0284 - accuracy: 0.9911
Test accuracy: 0.991
